`https://tech.ssut.me/understanding-python-metaclasses/`, `https://wikidocs.net/21056`내용정리

## 클래스를 객체로
대부분 언어에서 클래스는 어떻게 객체를 생성할지에 대해 정의하는 코드조각.  
Python에서 클래스는 그 이상의 의미를 갖는다. 클래스는 객체이기도 하다는 것.

In [1]:
class ObjectCreator:
    pass

my_object = ObjectCreator()
print(my_object)

- class 키워드를 사용할 때 python은 객체를 만들어낸다. 메모리에 ObjectCreator이라는 이름을 가진 객체가 생성된 것.  
- 이 객체(클래스)는 그 자체로 새로운 객체(인스턴스)를 만들 수 있으며, 이것이 왜 이 객체가 클래스인지를 나타낸다.
- 이 클래스는 객체로서
    1. 변수에 할당할 수 있고
    2. 복사할 수 잇고
    3. 새로운 속성을 추가할 수 있고
    4. 함수의 인자로 넘길 수 있다.

In [3]:
# 클래스가 객체이기 때문에 출력할 수 있다.
def echo(o):
    print(o)
    
echo(ObjectCreator)    # 함수의 인자로 넘길 수 있다.

<class '__main__.ObjectCreator'>


In [4]:
print(hasattr(ObjectCreator, 'new_attribute'))

False


In [5]:
ObjectCreator.new_attribute = 'foo'   # 클래스에 새로운 속성 추가.
print(hasattr(ObjectCreator, 'new_attribute'))

True


In [7]:
# 새로 생성되는 인스턴스도 추가된 새로운 속성을 갖는다.
instance = ObjectCreator()
print(hasattr(instance, 'new_attribute'))

True


In [9]:
ObjectCreatorMirror = ObjectCreator  # 클래스를 변수에 할당할 수 있다.
print(ObjectCreatorMirror.new_attribute)

foo


In [10]:
ObjectCreatorMirror

__main__.ObjectCreator

## 동적으로 클래스 생성하기

클래스가 객체이기 때문에 객체처럼 그때그때 새로운 클래스를 만들 수 있다.  

In [11]:
# class 키워드를 이용하여 함수에서 클래스 만들기
def choose_class(name):
    if name == 'foo':
        class Foo(object):
            pass
        return Foo     # 인스턴스가 아닌 클래스 반환
    else:
        class Bar:
            pass
        return Bar
    
MyClass = choose_class('foo')
print(MyClass)

<class '__main__.choose_class.<locals>.Foo'>


In [12]:
print(MyClass())  # 이 클래스로 새로운 객체를 만들 수 있다.

<__main__.choose_class.<locals>.Foo object at 0x06119390>


이 방법은 여전히 클래스 전체를 직접 작성해야 하기 때문에 동적이라 보기 어렵다.  
다른방법으로도 만들 수 있다!!!

#### 클래스를 만들어내는 type의 기능!!!!

In [13]:
print(type(1))

<class 'int'>


`type`의 쓰임..
```
type(name of the class, 
     tuple of the parent class (for inheritance, can be empty), 
     dictionary containing attributes names and values)
 ```

In [14]:
class MyShinyClass:
    pass

위 클래스는 다음과 같이 직접 생성될 수 있다.

In [15]:
MyShinyClass = type('MyShinyClass', (), {})   # 클래스 객체 반환.
print(MyShinyClass)

<class '__main__.MyShinyClass'>


인자로 넘긴 `MyShinyClass`가 클래스 이름으로 사용, 이를 할당한 변수는 클래스 레퍼런스를 갖고있다.  
`type`은 클래스 속성을 정의하기 위해 `dict`를 인자로 받는다.

In [16]:
class Foo:
    bar = True

In [17]:
# 위 코드와 동일
Foo = type('FOO', (), {'bar':True})

In [19]:
print(Foo)

<class '__main__.FOO'>


In [20]:
print(Foo.bar)

True


In [21]:
f = Foo()
print(f)

In [22]:
print(f.bar)

True


상속도 받을 수 있다.

In [23]:
class FooChild(Foo):
    pass

In [24]:
# 위와 동일..
FooChild = type('FooChild', (Foo,), {})
print(FooChild)

<class '__main__.FooChild'>


In [25]:
print(FooChild.bar) # bar is inherited from Foo

True


클래스에 메소드를 추가하고 싶을 때는 적절한 모양으로 함수를 만들고 인자로 넘기면 된다.

In [26]:
def echo_bar(self):
    print(self.bar)
    
FooChild = type('FooChild', (Foo,), {'echo_bar':echo_bar})
hasattr(Foo, 'echo_bar')

False

In [27]:
hasattr(FooChild, 'echo_bar')

True

In [28]:
my_foo = FooChild()
my_foo.echo_bar()

True


일반 클래스와 동일하게 새로운 메소드나 속성을 추가할 수도 있다.

In [29]:
def echo_bar_more(self):
    print('yet another method')
    
FooChild.echo_bar_more = echo_bar_more
hasattr(FooChild, 'echo_bar_more')

True

## 그래서..? metaclasses는 무엇인가
메타클래스는 클래스를 만드는 `무언가`이다.  
우리는 객체를 만들기 위해 클래스를 정의한다.  
하지만 Python에서 클래스는 곧 객체다.  
메타클래스는 이러한 객체를 만드는 `무언가`이다.  
메타클래스는 `클래스의 클래스`이며, 다음과 같은 방법으로 묘사된다.

```python
MyClass = MetaClass()
MyObject = MyClass()
```

위에서 배운대로 `type`을 통해 정의할 수도 있다.

```python
MyClass = type('MyClass', (), {})
```

여기서 `type`함수가 실제로 메타클래스!  
`type`은 Python이 실제로 보이는 코드 뒤에서 클래스를 생성하는 메타클래스이다.  
